<a href="https://colab.research.google.com/github/ASBTharaka/Music_AI/blob/main/Music.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install music21

In [3]:
import pandas as pd


In [4]:
df=pd.read_csv("/content/drive/MyDrive/Music/train/chorale_000.csv")
df

,note0,note1,note2,note3
0,74,70,65,58
1,74,70,65,58
2,74,70,65,58
3,74,70,65,58
4,75,70,58,55
...,...,...,...,...
187,70,65,62,46
188,70,65,62,46
189,70,65,62,46
190,70,65,62,46


In [6]:
import os

# Base path in Google Drive
base_path = "/content/drive/MyDrive/Music"

train_dir = os.path.join(base_path, "train")
valid_dir = os.path.join(base_path, "valid")
test_dir  = os.path.join(base_path, "test")

train_files = sorted([os.path.join(train_dir, f) for f in os.listdir(train_dir) if f.endswith('.csv')])
valid_files = sorted([os.path.join(valid_dir, f) for f in os.listdir(valid_dir) if f.endswith('.csv')])
test_files  = sorted([os.path.join(test_dir, f)  for f in os.listdir(test_dir)  if f.endswith('.csv')])


In [8]:
train_data=[pd.read_csv(f).values.tolist() for f in train_files]
valid_data=[pd.read_csv(f).values.tolist() for f in valid_files]
test_data=[pd.read_csv(f).values.tolist() for f in test_files]



In [9]:
from music21 import stream, chord

Music = train_data[20]
s = stream.Stream()
for row in Music:
    s.append(chord.Chord([n for n in row if n], quarterLength=1))
s.show('midi')

In [10]:
#Lowest note: 36 -> C1
#Highest note: 81 -> A5
#Silence: 0

#preprocessing
import numpy as np

min_note, max_note = 36, 81
window_size, window_offset, batch_size = 32, 16, 32


def make_xy(Music):
    # get segments of 33 chords with 16 chords offset between them
    windows = [c[i:i + window_size + 1] for c in Music for i in range(0, len(c) - window_size, window_offset)]

    data = np.array(windows, dtype=int)
    # if note is 0, keep it, otherwise rescale notes from 36-81 to 1-46
    data = np.where(data==0, 0, data - min_note + 1)
    # make the range 0-46 in total
    data = np.clip(data, 0, max_note - min_note + 1)

    flat = data.reshape(data.shape[0], -1)

    # return every chord except the last one for X (32 notes) and everything except the first one for Y (32 notes)
    return flat[:, :-1], flat[:, 1:]


X_train, Y_train = make_xy(train_data)
X_valid, Y_valid = make_xy(valid_data)
X_test,  Y_test  = make_xy(test_data)


In [11]:
X_train.shape

(3111, 131)

In [12]:
Y_train.shape

(3111, 131)

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Conv1D, BatchNormalization, LSTM, Dropout
from tensorflow.keras.optimizers import Nadam

In [14]:
train_notes = set([z for x in train_data for y in x for z in y])
test_notes = set([z for x in test_data for y in x for z in y])
valid_notes = set([z for x in valid_data for y in x for z in y])

num_notes = len(set.union(train_notes, test_notes, valid_notes))
num_notes

47

In [15]:
model = Sequential()

# Embedding layer so the model can learn representations of the notes (learns vector representation of notes)
# Integers have no geometry, by learning embeddings, we can keep "nearby" notes close in vector space (e.g., same pitch-class, close octaves etc.)
# Since we don't have too many notes, we can use a small dimensionality like 5
model.add(Embedding(input_dim=num_notes, output_dim=5, input_shape=[None]))
# 1D convs allow us to extract temporal patterns in parallel (unlike RNN layers)
# 1D convs slide 1D kernels / filters over our feature vector to learn temporal patterns
# Padding causal means we cannot look ahead, so we keep causality
model.add(Conv1D(32, kernel_size=2, padding="causal", activation="relu"))  # here 32 filters of size 2
# Batch norms after each conv keep activations well-scaled and consistent across the whole stack of layers
# Counteracts vanishing / exploding gradients, allows for higher stable learning rates and faster training
model.add(BatchNormalization())
# Dilation rate means how much we look back
# For example kernel size 2 and dilation rate of one means look at t and t-1
# Kernel size 2 and dilation rate of two means look at t and t-2
# Kernel size 2 and dilation rate of 16 means look at t and t-16
# Stacking these increasing dilation rates allows us to efficiently cover short, medium and longer history
# We grow the receptive field without using a lot of model parameters
# We indirectly connect notes that are up to 32 positions apart
model.add(Conv1D(48, kernel_size=2, padding="causal", activation="relu", dilation_rate=2))
model.add(BatchNormalization())
# Also, since we increase dilation rate, each conv layer sees a wider time span
# More kernels / filters allow us to capture more kinds of patterns
# If we don't increase this, we could end up with a bottleneck here
model.add(Conv1D(64, kernel_size=2, padding="causal", activation="relu", dilation_rate=4))
model.add(BatchNormalization())
model.add(Conv1D(96, kernel_size=2, padding="causal", activation="relu", dilation_rate=8))
model.add(BatchNormalization())
model.add(Conv1D(128, kernel_size=2, padding="causal", activation="relu", dilation_rate=16))
model.add(BatchNormalization())
# Just a bit of regularization here so the model does not rely too much on individual features
model.add(Dropout(0.05))
# The conv layers summarized local and mid-range context into richer features for us
# The LSTM now only has to track longer structure of the music
# Doing LSTM first would lead to discovering local AND long patterns, which is slower and harder to optimize
# Convs first is cheaper and parallelizable
# We can basically combine the layers to look back 32 tokens (like a binary system numbers)
model.add(LSTM(256, return_sequences=True))
# Finally a dense layer to project the LSTM output to logits for each possible note
model.add(Dense(num_notes, activation='softmax'))

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:100: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, None, 5)        │           235 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, None, 32)       │           352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, None, 32)       │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, None, 48)       │         3,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, None, 48)       │           192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, None, 64)       │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, None, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, None, 96)       │        12,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, None, 96)       │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, None, 128)      │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, None, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, None, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, None, 256)      │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, None, 47)       │        12,079 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 454,794 (1.73 MB)

 Trainable params: 454,058 (1.73 MB)

 Non-trainable params: 736 (2.88 KB)

In [16]:
optimizer = Nadam(learning_rate=1e-3)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=20, validation_data=[X_valid, Y_valid], batch_size=batch_size)

Epoch 1/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 61s 554ms/step - accuracy: 0.3200 - loss: 2.6337 - val_accuracy: 0.0785 - val_loss: 3.7808
Epoch 2/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 81s 545ms/step - accuracy: 0.7516 - loss: 0.9414 - val_accuracy: 0.0992 - val_loss: 4.0371
Epoch 3/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 58s 590ms/step - accuracy: 0.7930 - loss: 0.7449 - val_accuracy: 0.1597 - val_loss: 3.8526
Epoch 4/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 78s 548ms/step - accuracy: 0.8088 - loss: 0.6650 - val_accuracy: 0.2312 - val_loss: 3.1572
Epoch 5/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 58s 593ms/step - accuracy: 0.8199 - loss: 0.6115 - val_accuracy: 0.3175 - val_loss: 2.5169
Epoch 6/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 78s 551ms/step - accuracy: 0.8302 - loss: 0.5666 - val_accuracy: 0.4963 - val_loss: 1.7158
Epoch 7/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 82s 555ms/step - accuracy: 0.8385 - loss: 0.5346 - val_accuracy: 0.7838 - val_loss: 0.7428
Epoch 8/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 53s 542ms/step - accuracy: 0.8479 - loss: 0.4989 - val_accu

In [21]:
import numpy as np


def sample_next_note(probs):
    probabilities = np.asarray(probs, dtype=float)  # probabilities for each note to be the next

    probs_sum = probabilities.sum()  # get the sum for normalization

    # if the probability sum is zero, negative or infinite -> just return the note with the highest probability
    if probs_sum <= 0 or not np.isfinite(probs_sum):
        return int(np.argmax(probabilities))

    probabilities /= probs_sum # otherwise normalize the probabilities to be between 0 and 1
    return np.random.choice(len(probabilities), p=probabilities)  # randomly select a note based on probability


def generate_chorale(model, seed_chords, length):
    token_sequence = np.array(seed_chords, dtype=int)  # get starting chords / notes
    token_sequence = np.where(token_sequence == 0, token_sequence, token_sequence - 36 + 1)  # map all notes to 0-46 as in training
    token_sequence = token_sequence.reshape(1, -1)

    # we generate note by note, not chord by chord
    for _ in range(length * 4):
        next_token_probabilities = model.predict(token_sequence, verbose=0)[0, -1]  # get probabilities for next note from model
        next_token = sample_next_note(next_token_probabilities)  # sample from probabilities with sample function
        token_sequence = np.concatenate([token_sequence, [[next_token]]], axis=1)

    token_sequence = np.where(token_sequence == 0, token_sequence, token_sequence + 36 - 1)   # map to MIDI (0 & 36-81)

    return token_sequence.reshape(-1, 4)

In [22]:
# Initial chords (seed)
seed_chords = test_data[2][:8]

Music = seed_chords
s = stream.Stream()
for row in Music:
    s.append(chord.Chord([n for n in row if n], quarterLength=1))
s.show('midi')

In [23]:
# Complete actual chorale (ground truth)
seed_chords = test_data[2]

Music = seed_chords
s = stream.Stream()
for row in Music:
    s.append(chord.Chord([n for n in row if n], quarterLength=1))
s.show('midi')

In [25]:
# Generate new chords based on initial two chords (8 notes)
# Results can be better and more creative by introducing temperature, top_p etc.
seed_Music = test_data[2][:8]
new_Music =generate_chorale(model, seed_chords, 56)

In [28]:
new_Music

array([[73, 68, 61, 53],
       [73, 68, 61, 53],
       [73, 68, 61, 53],
       ...,
       [71, 67, 56, 49],
       [71, 66, 56, 49],
       [71, 66, 56, 49]])

In [30]:
# listen to generated piece

Music = new_Music.tolist()
s = stream.Stream()
for row in Music:
    s.append(chord.Chord([n for n in row if n], quarterLength=1))
s.show('midi')

In [32]:
import numpy as np

def generate_random_music(length, rest_probability=0.2, pitch_low=36, pitch_high=81, seed=None):
    rng = np.random.default_rng(seed)  # random number generator
    random_pitches = rng.integers(pitch_low, pitch_high + 1, size=(length, 4))  # generate random notes

    # some masking to have both silence and random pitches
    rest_mask = rng.random((length, 4)) < float(rest_probability)
    music = np.where(rest_mask, 0, random_pitches).astype(int)

    return music


In [33]:
# listen to completely random music to compare the quality to what our model generated
Music= generate_random_music(56).tolist()
s = stream.Stream()
for row in Music:
    s.append(chord.Chord([n for n in row if n], quarterLength=1))
s.show('midi')

In [34]:
# save model
model.save('bach_generation_conv1d_lstm.keras')

In [35]:
from tensorflow import keras

loaded_model = keras.models.load_model("bach_generation_conv1d_lstm.keras")

In [37]:
new_Music = generate_chorale(loaded_model, seed_chords, 56)

In [39]:

new_Music

array([[73, 68, 61, 53],
       [73, 68, 61, 53],
       [73, 68, 61, 53],
       ...,
       [78, 66, 59, 59],
       [78, 66, 59, 59],
       [76, 69, 59, 47]])

In [40]:
Music= generate_random_music(56).tolist()
s = stream.Stream()
for row in Music:
    s.append(chord.Chord([n for n in row if n], quarterLength=1))
s.show('midi')